In [178]:
import pandas as pd
import pyodbc
import sqlite3

# Connection

In [179]:
DB = {"servername": "MSI\\SQLEXPRESS",
      "database": "DEDSDatabase1"}

export_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER=' + DB['servername'] + 
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

In [180]:
def select_from(name, connection):
      dataframe = pd.read_sql_query("SELECT * FROM " + name, connection)
      
      columni = {}
      for column in dataframe.columns:
            columni[column] = name.upper() + "_" + column.lower()
      dataframe.rename(columns=columni, inplace=True)
      return dataframe
      

def find_mutual_columns(dataset1, dataset2):
      column_names = set([])
      mutual_column_names = []
      for column in dataset1:
            column_names.add(column)
      for column in dataset2:
            if column in column_names:
                  mutual_column_names.append(column)
      return mutual_column_names

def merge_differing_columns_simple(dataframe1, dataframe2, key):
      # Pak de unieke columns uit de tweede dataframe
      column_names1 = set(dataframe1.columns)
      column_names2 = set(dataframe2.columns)
      uniques_from_2 = [key]
      for column in column_names2:
            if column not in column_names1:
                  uniques_from_2.append(column)
      
      # Merge deze met de eerste dataframe
      return pd.merge(dataframe1, dataframe2.loc[:, uniques_from_2], on=key, how='left')

def merge_differing_columns(dataframe1, dataframe2, left_key, right_key):
      # Pak de unieke columns uit de tweede dataframe
      column_names1 = set(dataframe1.columns)
      column_names2 = set(dataframe2.columns)
      uniques_from_2 = [right_key]
      for column in column_names2:
            if column not in column_names1:
                  uniques_from_2.append(column)
      
      # Merge deze met de eerste dataframe
      return pd.merge(dataframe1, dataframe2.loc[:, uniques_from_2], left_on=left_key, right_on=right_key, how='left')

In [181]:
connectie_sales = sqlite3.connect('go_sales.sqlite')
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"

product = select_from("product", connectie_sales)
product_type = select_from("product_type", connectie_sales)
product_line = select_from("product_line", connectie_sales)
SALES_sales_staff = select_from("sales_staff", connectie_sales)
SALES_sales_branch = select_from("sales_branch", connectie_sales)
SALES_retailer_site = select_from("retailer_site", connectie_sales)
SALES_country = select_from("country", connectie_sales)
order_header = select_from("order_header", connectie_sales)
order_details = select_from("order_details", connectie_sales)
order_method = select_from("order_method", connectie_sales)
target = select_from("SALES_TARGETData", connectie_sales)
returned_item = select_from("returned_item", connectie_sales)
return_reason = select_from("return_reason", connectie_sales)
sales_target = select_from("SALES_TARGETData", connectie_sales)

In [182]:
connectie_staff = sqlite3.connect('go_staff.sqlite')

course = select_from("course", connectie_staff)
STAFF_sales_staff = select_from("sales_staff", connectie_staff)
STAFF_sales_branch = select_from("sales_branch", connectie_staff)
satisfaction = select_from("satisfaction", connectie_staff)
satisfaction_type = select_from("satisfaction_type", connectie_staff)
training = select_from("training", connectie_staff)

In [183]:
connectie_crm = sqlite3.connect('go_crm.sqlite')

age_group = select_from("age_group", connectie_crm)
CRM_country = select_from("country", connectie_crm)
retailer = select_from("retailer", connectie_crm)
retailer_contact = select_from("retailer_contact", connectie_crm)
retailer_headquarters = select_from("retailer_headquarters", connectie_crm)
retailer_segment = select_from("retailer_segment", connectie_crm)
CRM_retailer_site = select_from("retailer_site", connectie_crm)
retailer_type = select_from("retailer_type", connectie_crm)
sales_demographic = select_from("sales_demographic", connectie_crm)
sales_territory = select_from("sales_territory", connectie_crm)

In [184]:
# CSV files
inventory_levels = pd.read_csv('GO_SALES_INVENTORY_LEVELSData.csv')
product_forecast = pd.read_csv('GO_SALES_PRODUCT_FORECASTData.csv')

In [185]:
# Merge similar tables
sales_staff = merge_differing_columns_simple(STAFF_sales_staff, SALES_sales_staff, 'SALES_STAFF_sales_staff_code')
sales_branch = merge_differing_columns_simple(SALES_sales_branch, STAFF_sales_branch, 'SALES_BRANCH_sales_branch_code')
country = merge_differing_columns_simple(SALES_country, CRM_country, 'COUNTRY_country_code')
retailer_site = merge_differing_columns_simple(SALES_retailer_site, CRM_retailer_site, 'RETAILER_SITE_retailer_site_code')

# Nina

In [186]:
# PRODUCT
output_PRODUCT = pd.merge(product, product_type, left_on='PRODUCT_product_type_code', right_on='PRODUCT_TYPE_product_type_code', how='left')
output_PRODUCT.drop('PRODUCT_TYPE_product_type_code', axis=1, inplace=True)
output_PRODUCT = pd.merge(output_PRODUCT, product_line, left_on='PRODUCT_TYPE_product_line_code', right_on='PRODUCT_LINE_product_line_code', how='left')
output_PRODUCT.drop('PRODUCT_LINE_product_line_code', axis=1, inplace=True)

# RETURN_REASON
output_RETURN_REASON = return_reason

# SALES_TARGETData
output_SALES_TARGETData = sales_target

# PRODUCT_FORECASTData
output_PRODUCT_FORECASTData = product_forecast

# INVENTORY_LEVELSData
output_INVENTORY_LEVELSData = inventory_levels

# Yemeserach